In [1]:
%pip install scikit-learn llama-index llama-index-embeddings-ollama pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# 🔍 Demonstration: Retrieval-Augmented Generation (RAG) with Cosine Similarity
This notebook showcases how Retrieval-Augmented Generation (RAG) works under the hood by using cosine similarity to retrieve the most relevant text chunks in response to a query.

## 🧾 Test Document
The evaluation is based on a synthetic insurance document generated by GPT-4o (OpenAI). The document includes:

- A fictional insurance contract by Allianz AG
- Customer details (e.g., policyholder: Max Mustermann)
- Key contractual clauses and policy terms
- An unrelated paragraph to simulate noise or misleading context (for testing retrieval robustness)

## 🧠 Embedding Models Tested
To compute similarity and retrieve relevant text chunks, we use vector embeddings from the following models:

- text-embedding-ada-002 (OpenAI)
- Llama3.3-70B (via embedding interface)

## 🧠 Language Synthetization with LLM
To generate an answer, we retrieve the Top-K from the similarity scores, put them with the query in a meaningful context, and pass the context to the LLM:

- GPT-3.5-Turbo-Instruct
- Llama3.3-70B

## ⚙️ Workflow Overview
1. The document is split into manageable text chunks
2. Each chunk is embedded into a vector space
3. A user query is embedded and compared to all chunk vectors using cosine similarity
4. The top-k most relevant chunks are passed to the LLM (Llama3.3:70B or GPT-3.5-Instruct) for response generation

This setup helps illustrate how RAG systems combine information retrieval with language generation to provide grounded and context-aware responses.

In [101]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.settings import Settings

import os

api_key = os.getenv('OPENAI_KEY')

llm = OpenAI(
    temperature=0,
    api_key=api_key,
    model="gpt-3.5-turbo-instruct"
)
embed_model = OpenAIEmbedding(
    model='text-embedding-ada-002',
    api_key=api_key,
)
Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size=256

This notebook demonstrates the functionality of Retrieval Augmented Generation works in the background with the cosine similarity formula
Tested on a generated document by GPT-4o of OpenAI (the link is that.), the document contains exampled generated insurance contract of Allianz Ag, an insurance taker name, the policy, and a paragraph that has something to do with insurance but is not related to the document.
Also tested on embed model of llama3.3-70b, gpt-3.5-instruct and text-embedding-ada-002 of openai for combining the top_k with the llm of llama and gpt-3.5-instruct.

In [73]:
from llama_index.core.readers import SimpleDirectoryReader

documents = SimpleDirectoryReader(input_files=['./data/insurance_contract_example.txt']).load_data(show_progress=True)

Loading files: 100%|██████████| 1/1 [00:00<00:00, 975.19it/s]


In [74]:
from llama_index.core.node_parser import SentenceSplitter

parser = SentenceSplitter(
    chunk_size=256,
    chunk_overlap=20,
)
chunks = parser.get_nodes_from_documents(documents)
len(chunks)

5

In [75]:
query = "Wer ist der Versicherer und wer ist Versicherungsnehmer? Was wird gehaftet?"
embeddings_document = [
    embed_model.get_text_embedding(chunk.text) for chunk in chunks
]
embedding_query = embed_model.get_text_embedding(query)

In [76]:
from sklearn.metrics.pairwise import cosine_similarity

similarities = cosine_similarity([embedding_query], embeddings_document)[0]

In [77]:
similarities

array([0.79948033, 0.84596817, 0.84828211, 0.85467669, 0.81015427])

In [78]:
import pandas as pd

pd.set_option('display.max_colwidth', None)
df = pd.DataFrame([similarities, [chunk.text for chunk in chunks]], index=["Similarity score", "Text"])
df.T

,Similarity score,Text
0,0.79948,"📉 Semantisch irrelevanter Abschnitt (für Demonstration)\nDie durchschnittliche Bearbeitungszeit von Leistungsanträgen im Bereich Zahnzusatzversicherung beträgt laut interner Statistik der Allianz AG etwa 7,3 Werktage, wobei regionale Unterschiede durch standortbezogene Ressourcenverteilung entstehen können.\n\nAllianz Versicherungs-AG\nKöniginstraße 28\n80802 München\nTelefon: 089 3800-0\nE-Mail: service@allianz.de\nWeb: www.allianz.de\n\nVersicherungsschein (Police-Nr."
1,0.845968,"1234 5678 9012)\nVertragsbeginn: 01.07.2025\nVersicherungsnehmer:\nMax Mustermann\nMusterstraße 12\n12345 Musterstadt\nGeburtsdatum: 01.01.1980\nVersichert seit: 01.07.2025\n\n🛡️ Vertragsübersicht\nVersicherungsart:\nPrivathaftpflichtversicherung – Komfortschutz\n\nGeltungsbereich:\nWeltweit (inkl. vorübergehende Auslandsaufenthalte bis 12 Monate)\n\nVersicherungssumme:\n10.000.000 € pauschal für Personen-, Sach- und Vermögensschäden\n\nSelbstbeteiligung:\n150 € je Schadensfall\n\nJährlicher Beitrag:\n120,00 € (inkl."
2,0.848282,"Versicherungssteuer)\n\nZahlweise:\njährlich, per SEPA-Lastschrift\n\nZahlungstermin:\n01.07. jeden Jahres\n\n📋 Wichtige Vertragsbedingungen\nWir, die Versicherer Allianz Versicherungs-AG, haften für den Versicherungsnehmer Max Mustermann im Rahmen der vereinbarten Bedingungen und Versicherungssummen gegenüber Dritten für Personen-, Sach- und Vermögensschäden, die durch ihn verursacht werden.\n\nDer Versicherer haftet für den Versicherungsnehmer Max Mustermann, sofern Ansprüche Dritter auf gesetzlicher Grundlage entstehen und im Rahmen der Privathaftpflichtversicherung abgedeckt sind."
3,0.854677,"Die Versicherung haftet für folgende Objekte, sofern diese sich im Eigentum oder rechtmäßigen Besitz des Versicherungsnehmers befinden und im Rahmen des Versicherungsvertrages eingeschlossen sind:\n\nWohnräume und Nebengebäude am Wohnort\n\nHaustiere (z. B. Hunde, Katzen – keine gefährlichen Tiere i.S.d. Gesetzes)\n\nBewegliche Gegenstände des täglichen Lebens\n\nSchäden im Rahmen gesetzlicher Haftung im privaten Umfeld\n\nDer Versicherungsschutz beginnt mit dem in der Police genannten Datum, sofern der erste Beitrag rechtzeitig gezahlt wurde.\n\nDie Vertragslaufzeit beträgt 1 Jahr und verlängert sich automatisch, sofern nicht 3 Monate vor Ablauf gekündigt wird.\n\nEs gelten die Allgemeinen Versicherungsbedingungen (AVB PHV 2025)."
4,0.810154,"📌 Kontakt bei Schadenmeldung\nTelefonische Schadenmeldung:\n089 3800-5555 (Mo–Fr, 8–20 Uhr)\n\nOnline:\nwww.allianz.de/schaden-melden\n\nE-Mail:\nschaden@allianz.de\n\n🖊️ Bestätigung\nBitte prüfen Sie die Angaben zu Ihrer Versicherung sorgfältig. Bei Fragen oder Änderungswünschen wenden Sie sich gerne an unseren Kundenservice.\n\nMit freundlichen Grüßen\nIhre Allianz Versicherungs-AG\n\nUnterschrift maschinell erstellt – gültig ohne Unterschrift\n\n🔒 Datenschutz-Hinweis\nIhre personenbezogenen Daten werden gemäß den geltenden Datenschutzvorschriften, insbesondere der DSGVO, verarbeitet. Weitere Informationen finden Sie unter www.allianz.de/datenschutz.\n\nHinweis: Dieses Dokument ist ein fiktives Beispiel und dient ausschließlich Demonstrationszwecken. Es besteht kein tatsächlicher Versicherungsvertrag."


In [79]:
sorted = df.T.sort_values(by=["Similarity score"], ascending=False)
sorted

,Similarity score,Text
3,0.854677,"Die Versicherung haftet für folgende Objekte, sofern diese sich im Eigentum oder rechtmäßigen Besitz des Versicherungsnehmers befinden und im Rahmen des Versicherungsvertrages eingeschlossen sind:\n\nWohnräume und Nebengebäude am Wohnort\n\nHaustiere (z. B. Hunde, Katzen – keine gefährlichen Tiere i.S.d. Gesetzes)\n\nBewegliche Gegenstände des täglichen Lebens\n\nSchäden im Rahmen gesetzlicher Haftung im privaten Umfeld\n\nDer Versicherungsschutz beginnt mit dem in der Police genannten Datum, sofern der erste Beitrag rechtzeitig gezahlt wurde.\n\nDie Vertragslaufzeit beträgt 1 Jahr und verlängert sich automatisch, sofern nicht 3 Monate vor Ablauf gekündigt wird.\n\nEs gelten die Allgemeinen Versicherungsbedingungen (AVB PHV 2025)."
2,0.848282,"Versicherungssteuer)\n\nZahlweise:\njährlich, per SEPA-Lastschrift\n\nZahlungstermin:\n01.07. jeden Jahres\n\n📋 Wichtige Vertragsbedingungen\nWir, die Versicherer Allianz Versicherungs-AG, haften für den Versicherungsnehmer Max Mustermann im Rahmen der vereinbarten Bedingungen und Versicherungssummen gegenüber Dritten für Personen-, Sach- und Vermögensschäden, die durch ihn verursacht werden.\n\nDer Versicherer haftet für den Versicherungsnehmer Max Mustermann, sofern Ansprüche Dritter auf gesetzlicher Grundlage entstehen und im Rahmen der Privathaftpflichtversicherung abgedeckt sind."
1,0.845968,"1234 5678 9012)\nVertragsbeginn: 01.07.2025\nVersicherungsnehmer:\nMax Mustermann\nMusterstraße 12\n12345 Musterstadt\nGeburtsdatum: 01.01.1980\nVersichert seit: 01.07.2025\n\n🛡️ Vertragsübersicht\nVersicherungsart:\nPrivathaftpflichtversicherung – Komfortschutz\n\nGeltungsbereich:\nWeltweit (inkl. vorübergehende Auslandsaufenthalte bis 12 Monate)\n\nVersicherungssumme:\n10.000.000 € pauschal für Personen-, Sach- und Vermögensschäden\n\nSelbstbeteiligung:\n150 € je Schadensfall\n\nJährlicher Beitrag:\n120,00 € (inkl."
4,0.810154,"📌 Kontakt bei Schadenmeldung\nTelefonische Schadenmeldung:\n089 3800-5555 (Mo–Fr, 8–20 Uhr)\n\nOnline:\nwww.allianz.de/schaden-melden\n\nE-Mail:\nschaden@allianz.de\n\n🖊️ Bestätigung\nBitte prüfen Sie die Angaben zu Ihrer Versicherung sorgfältig. Bei Fragen oder Änderungswünschen wenden Sie sich gerne an unseren Kundenservice.\n\nMit freundlichen Grüßen\nIhre Allianz Versicherungs-AG\n\nUnterschrift maschinell erstellt – gültig ohne Unterschrift\n\n🔒 Datenschutz-Hinweis\nIhre personenbezogenen Daten werden gemäß den geltenden Datenschutzvorschriften, insbesondere der DSGVO, verarbeitet. Weitere Informationen finden Sie unter www.allianz.de/datenschutz.\n\nHinweis: Dieses Dokument ist ein fiktives Beispiel und dient ausschließlich Demonstrationszwecken. Es besteht kein tatsächlicher Versicherungsvertrag."
0,0.79948,"📉 Semantisch irrelevanter Abschnitt (für Demonstration)\nDie durchschnittliche Bearbeitungszeit von Leistungsanträgen im Bereich Zahnzusatzversicherung beträgt laut interner Statistik der Allianz AG etwa 7,3 Werktage, wobei regionale Unterschiede durch standortbezogene Ressourcenverteilung entstehen können.\n\nAllianz Versicherungs-AG\nKöniginstraße 28\n80802 München\nTelefon: 089 3800-0\nE-Mail: service@allianz.de\nWeb: www.allianz.de\n\nVersicherungsschein (Police-Nr."


In [80]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents=chunks, embed_model=embed_model)

In [84]:
from IPython.display import display

top_k = 1 # default is 2, can be extended or reduced!

query = "Wer ist der Versicherer und wer der Versicherungsnehmer? Was wird gehaftet?"
query_engine = index.as_query_engine(similarity_top_k=top_k, llm=llm)
response = query_engine.query(query)

In [85]:
pd.DataFrame([[query], [response.response]], index=['Query', 'Output']).T

,Query,Output
0,Wer ist der Versicherer und wer der Versicherungsnehmer? Was wird gehaftet?,"\nDer Versicherer ist nicht explizit genannt, da es sich um ein Beispiel handelt. Der Versicherungsnehmer ist die Person, die den Vertrag abschließt und somit die Versicherung in Anspruch nehmen kann. Es wird für Wohnräume, Nebengebäude, Haustiere, bewegliche Gegenstände und Schäden im privaten Umfeld gehaftet."


In [86]:
from IPython.display import display

top_k = 2 # default is 2, can be extended or reduced!

query = "Wer ist der Versicherer und wer der Versicherungsnehmer? Was wird gehaftet?"
query_engine = index.as_query_engine(similarity_top_k=top_k)
response = query_engine.query(query)

In [87]:
pd.DataFrame([[query], [response.response]], index=['Query', 'Output']).T

,Query,Output
0,Wer ist der Versicherer und wer der Versicherungsnehmer? Was wird gehaftet?,"\nDer Versicherer ist die Allianz Versicherungs-AG und der Versicherungsnehmer ist Max Mustermann. Der Versicherer haftet für den Versicherungsnehmer Max Mustermann im Rahmen der vereinbarten Bedingungen und Versicherungssummen gegenüber Dritten für Personen-, Sach- und Vermögensschäden, die durch ihn verursacht werden."


In [88]:
%pip install llama-index-llms-ollama

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.core.settings import Settings

import os

base_url = os.getenv('OLLAMA_URL', 'http://localhost:11434')

llm = Ollama(
    model='llama3.3:70b',
    base_url=base_url,
    temperature=0,
    request_timeout=42069,
)
embed_model = OllamaEmbedding(
    model_name='llama3.3:70b',
    base_url=base_url
)
Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size=256

In [90]:
from llama_index.core.readers import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter

documents = SimpleDirectoryReader(input_files=['./data/insurance_contract_example.txt']).load_data(show_progress=True)

parser = SentenceSplitter(
    chunk_size=256,
    chunk_overlap=20,
)
chunks = parser.get_nodes_from_documents(documents)
len(chunks)

Loading files: 100%|██████████| 1/1 [00:00<00:00, 1484.71it/s]


5

In [91]:
query = "Wer ist der Versicherer und wer ist Versicherungsnehmer? Was wird gehaftet?"
embeddings_document = [
    embed_model.get_text_embedding(chunk.text) for chunk in chunks
]
embedding_query = embed_model.get_text_embedding(query)

In [92]:
from sklearn.metrics.pairwise import cosine_similarity

similarities = cosine_similarity([embedding_query], embeddings_document)[0]

In [94]:
import pandas as pd

pd.set_option('display.max_colwidth', None)
df = pd.DataFrame([similarities, [chunk.text for chunk in chunks]], index=["Similarity score", "Text"])
df.T.sort_values(by=["Similarity score"], ascending=False)

,Similarity score,Text
2,0.746873,"Versicherungssteuer)\n\nZahlweise:\njährlich, per SEPA-Lastschrift\n\nZahlungstermin:\n01.07. jeden Jahres\n\n📋 Wichtige Vertragsbedingungen\nWir, die Versicherer Allianz Versicherungs-AG, haften für den Versicherungsnehmer Max Mustermann im Rahmen der vereinbarten Bedingungen und Versicherungssummen gegenüber Dritten für Personen-, Sach- und Vermögensschäden, die durch ihn verursacht werden.\n\nDer Versicherer haftet für den Versicherungsnehmer Max Mustermann, sofern Ansprüche Dritter auf gesetzlicher Grundlage entstehen und im Rahmen der Privathaftpflichtversicherung abgedeckt sind."
3,0.696489,"Die Versicherung haftet für folgende Objekte, sofern diese sich im Eigentum oder rechtmäßigen Besitz des Versicherungsnehmers befinden und im Rahmen des Versicherungsvertrages eingeschlossen sind:\n\nWohnräume und Nebengebäude am Wohnort\n\nHaustiere (z. B. Hunde, Katzen – keine gefährlichen Tiere i.S.d. Gesetzes)\n\nBewegliche Gegenstände des täglichen Lebens\n\nSchäden im Rahmen gesetzlicher Haftung im privaten Umfeld\n\nDer Versicherungsschutz beginnt mit dem in der Police genannten Datum, sofern der erste Beitrag rechtzeitig gezahlt wurde.\n\nDie Vertragslaufzeit beträgt 1 Jahr und verlängert sich automatisch, sofern nicht 3 Monate vor Ablauf gekündigt wird.\n\nEs gelten die Allgemeinen Versicherungsbedingungen (AVB PHV 2025)."
4,0.606278,"📌 Kontakt bei Schadenmeldung\nTelefonische Schadenmeldung:\n089 3800-5555 (Mo–Fr, 8–20 Uhr)\n\nOnline:\nwww.allianz.de/schaden-melden\n\nE-Mail:\nschaden@allianz.de\n\n🖊️ Bestätigung\nBitte prüfen Sie die Angaben zu Ihrer Versicherung sorgfältig. Bei Fragen oder Änderungswünschen wenden Sie sich gerne an unseren Kundenservice.\n\nMit freundlichen Grüßen\nIhre Allianz Versicherungs-AG\n\nUnterschrift maschinell erstellt – gültig ohne Unterschrift\n\n🔒 Datenschutz-Hinweis\nIhre personenbezogenen Daten werden gemäß den geltenden Datenschutzvorschriften, insbesondere der DSGVO, verarbeitet. Weitere Informationen finden Sie unter www.allianz.de/datenschutz.\n\nHinweis: Dieses Dokument ist ein fiktives Beispiel und dient ausschließlich Demonstrationszwecken. Es besteht kein tatsächlicher Versicherungsvertrag."
1,0.508678,"1234 5678 9012)\nVertragsbeginn: 01.07.2025\nVersicherungsnehmer:\nMax Mustermann\nMusterstraße 12\n12345 Musterstadt\nGeburtsdatum: 01.01.1980\nVersichert seit: 01.07.2025\n\n🛡️ Vertragsübersicht\nVersicherungsart:\nPrivathaftpflichtversicherung – Komfortschutz\n\nGeltungsbereich:\nWeltweit (inkl. vorübergehende Auslandsaufenthalte bis 12 Monate)\n\nVersicherungssumme:\n10.000.000 € pauschal für Personen-, Sach- und Vermögensschäden\n\nSelbstbeteiligung:\n150 € je Schadensfall\n\nJährlicher Beitrag:\n120,00 € (inkl."
0,0.390144,"📉 Semantisch irrelevanter Abschnitt (für Demonstration)\nDie durchschnittliche Bearbeitungszeit von Leistungsanträgen im Bereich Zahnzusatzversicherung beträgt laut interner Statistik der Allianz AG etwa 7,3 Werktage, wobei regionale Unterschiede durch standortbezogene Ressourcenverteilung entstehen können.\n\nAllianz Versicherungs-AG\nKöniginstraße 28\n80802 München\nTelefon: 089 3800-0\nE-Mail: service@allianz.de\nWeb: www.allianz.de\n\nVersicherungsschein (Police-Nr."


In [97]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents=chunks, embed_model=embed_model)

top_k = 1 # default is 2, can be extended or reduced!
query = "Wer ist der Versicherer und wer der Versicherungsnehmer? Was wird gehaftet?"
query_engine = index.as_query_engine(similarity_top_k=top_k, llm=llm)
response = query_engine.query(query)

In [98]:
pd.DataFrame([[query], [response.response]], index=['Query', 'Output']).T

,Query,Output
0,Wer ist der Versicherer und wer der Versicherungsnehmer? Was wird gehaftet?,"Der Versicherer ist die Allianz Versicherungs-AG und der Versicherungsnehmer ist Max Mustermann. Der Versicherer haftet für den Versicherungsnehmer gegenüber Dritten für Personen-, Sach- und Vermögensschäden, die durch ihn verursacht werden, sowie für Ansprüche Dritter auf gesetzlicher Grundlage im Rahmen der Privathaftpflichtversicherung."


In [99]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents=chunks, embed_model=embed_model)

top_k = 2 # default is 2, can be extended or reduced!
query = "Wer ist der Versicherer und wer der Versicherungsnehmer? Was wird gehaftet?"
query_engine = index.as_query_engine(similarity_top_k=top_k, llm=llm)
response = query_engine.query(query)

In [100]:
pd.DataFrame([[query], [response.response]], index=['Query', 'Output']).T

,Query,Output
0,Wer ist der Versicherer und wer der Versicherungsnehmer? Was wird gehaftet?,"Der Versicherer ist die Allianz Versicherungs-AG, während der Versicherungsnehmer Max Mustermann ist. Der Versicherer haftet für den Versicherungsnehmer gegenüber Dritten für Personen-, Sach- und Vermögensschäden, die durch ihn verursacht werden, sofern Ansprüche auf gesetzlicher Grundlage entstehen und im Rahmen der Privathaftpflichtversicherung abgedeckt sind."
